In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

Loading of whats-cooking dataset

In [11]:
curr_path = os.path.dirname(os.getcwd())
data_path = os.path.join(curr_path, 'data')
input_file = os.path.join(data_path, 'train.json')

In [12]:
curr_path

'/Users/gabrieladvorakova/School/Data_mining'

In [9]:
with open(input_file) as file:
   data = json.load(file)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/gabrieladvorakova/School/Data_mining/data/train.json'